This is a sample for celestial navigation for a stationary observer<br> 
© August Linnman, 2025, email: august@linnman.net<br>
MIT License (see [LICENSE file](LICENSE))

Jupyter notebook for ordinary three-fix celestial navigation (stationary).

Run it once to create the input form. Used data will be saved for later sessions. 


In [55]:
# Importing Python libraries
from time import time
from starfix import Sight, SightCollection, get_representation, get_google_map_string,\
                    IntersectError, LatLonGeodetic

In [ ]:
# Input form
import ipywidgets as widgets
from ipywidgets import VBox
import json

file_name = "notebook1-dict.json"

try:
    with open(file_name, "r") as f:
        s = f.read ()
        numDict = json.loads (s)
except FileNotFoundError:
    numDict = {"ObjectName1" : "Sun",
               "Altitude1" : "55:7:54.4",
               "Time1" : "2024-05-05 15:55:18+00:00",
               
               "ObjectName2" : "Sun",
               "Altitude2" : "19:28:27.4",
               "Time2" : "2024-05-05 23:01:19+00:00",
               
               "ObjectName3" : "Vega",
               "Altitude3" : "30:16:15.7",
               "Time3" : "2024-05-06 04:04:13+00:00",
               
               "DrpLat" : "40",
               "DrpLon" : "-90"}

def dump_dict ():
    jDump = json.dumps (numDict)
    with open(file_name, "w") as f:
        f.write(jDump)    

def handle_change (change):
    if change['type'] == 'change' and change['name'] == 'value':
        the_owner = change['owner']
        assert isinstance (the_owner, TextWidget)
        the_owner.handle_event (change)

class TextWidget (widgets.Text):

    def __init__ (self, attrName, description):
        self.__attrName = attrName
        super().__init__ (numDict[self.__attrName], description=description, disabled=False)
        self.observe (handle_change)

    def handle_event (self, change):
        numDict[self.__attrName] =change['new']
        dump_dict ()

widget_array = []
typeArray = ["ObjectName", "Altitude", "Time"]
labelArray = ["NAME", "ALT", "TIME"]
widget_array.append (TextWidget ("DrpLat","DRP_LAT"))
widget_array.append (TextWidget ("DrpLon","DRP_LON"))
for i in range (3):
    for j in range (3):
        widget_array.append (TextWidget (typeArray[j]+str(i+1),
                                         description=labelArray[j]+"_"+str(i+1)))

VBox (widget_array)

In [57]:
# SIGHTS

def get_starfixes (drp_pos : LatLonGeodetic) -> SightCollection :
    ''' Returns a list of used star fixes (SightCollection) '''

    Sight.set_estimated_position (drp_pos)
    retval = []
    for i in range (3):
        retval.append (Sight ( object_name          = numDict[typeArray[0]+str(i+1)],
                               measured_alt         = numDict[typeArray[1]+str(i+1)],
                               set_time             = numDict[typeArray[2]+str(i+1)],                
                             ))

    return SightCollection (retval)


In [ ]:
# SIGHT REDUCTION.

the_pos = LatLonGeodetic (float(numDict["DrpLat"]), 
                          float(numDict["DrpLon"])) # Rough DRP position

intersections = None
collection = None
the_map = None
try:
    intersections, _, _, collection =\
            SightCollection.get_intersections_conv (return_geodetic=True,
                                                    estimated_position=the_pos,
                                                    get_starfixes=get_starfixes,
                                                    assume_good_estimated_position=True)
    
    assert intersections is not None
    assert collection is not None
    print (get_representation(intersections,1))
    assert isinstance (intersections, LatLonGeodetic)
    print ("Google Map Coordinate = " + get_google_map_string(intersections,4))

except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve))
    if ve.coll_object is not None:
        if isinstance (ve.coll_object, SightCollection):
            collection = ve.coll_object

if isinstance (intersections, tuple):
    intersections = None

if collection is not None:
    the_map = collection.render_folium (intersections)
the_map